# DAY 3 : Function calling 

## Classifying teh question into a calss to reduce the functions/tools

### Supported models (https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm) 
- cohere.command-r-08-2024
- cohere.command-r-16k
- cohere.command-r-plus
- cohere.command-r-plus-08-2024
- meta.llama-3.1-405b-instruct
- meta.llama-3.1-70b-instruct
- meta.llama-3.2-90b-vision-instruct
  


### Set up variables

In [ ]:
# set up the  variables
from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import OnDemandServingMode, EmbedTextDetails,CohereChatRequest, ChatDetails
import oci

CONFIG_PROFILE = "DEFAULT"
compartmentId= "ocid1.compartment.oc1..aaaaaaaa5wmdeu3rf5s4rs4l66rksphne2orz4buauniiqtar63du6ni7icq"
llm_service_endpoint= "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
llm_client = None
llm_payload = None

### load the profile

In [ ]:
# oci key enabled for api access
config = oci.config.from_file('~/.oci/workshop', CONFIG_PROFILE)

In [4]:
### set up Chat Request

In [3]:


cohere_chat_request = CohereChatRequest()
cohere_chat_request.preamble_override = """
        
        You are a call classifier you carefully analyze teh question and classify it into one of the following categories. 
        you then return just the category in response
        
        categories: billing, outage, program, service
        
        eg: 
        question:  can i pay my bill by creditcard
        answer: billing
        question: i need to cancel my service as i am moving
        answer: service
        question: when will by power come back on?
        answer: outage
        
        """
cohere_chat_request.is_stream = False 
cohere_chat_request.max_tokens = 500
cohere_chat_request.temperature = 0.75
cohere_chat_request.top_p = 0.7
cohere_chat_request.frequency_penalty = 1.0


## Set up chat details

In [4]:

chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id="cohere.command-r-plus")
chat_detail.compartment_id = compartmentId
chat_detail.chat_request = cohere_chat_request

### get the LLM client 

In [5]:

# set up the LLM client 
llm_client = GenerativeAiInferenceClient(
                config=config,
                service_endpoint=llm_service_endpoint,
                retry_strategy=oci.retry.NoneRetryStrategy(),
                timeout=(10,240))

### Ask the question 

In [8]:
cohere_chat_request.message = "why is my bill so high"

llm_response = llm_client.chat(chat_detail)
llm_text = llm_response.data.chat_response.text

*** print response & citations

In [9]:
print("**************************Chat Result**************************")
#llm_text = llm_response.data.chat_response.text
print(llm_response.data.chat_response.text)

**************************Chat Result**************************
billing


## Exercise 


Merge your RAG Agent & Single step tool into same code and use clasiification to decide which to invoke
eg: 
1. If the queston is about weather, cll the single step tool
2. if the call is about batteries use the RAG to answer
3. if its about something else say you can only answer questions about weather or batteries

